Problem Statement

The COVID-19 Pandemic has disrupted learning for more than 56 million students in the United States. In the Spring of 2020, most states and local governments across the U.S. closed educational institutions to stop the spread of the virus. In response, schools and teachers have attempted to reach students remotely through distance learning tools and digital platforms. Until today, concerns of the exacaberting digital divide and long-term learning loss among America’s most vulnerable learners continue to grow.

Challenge

We challenge the Kaggle community to explore (1) the state of digital learning in 2020 and (2) how the engagement of digital learning relates to factors such as district demographics, broadband access, and state/national level policies and events.

We encourage you to guide the analysis with questions that are related to the themes that are described above (in bold font). Below are some examples of questions that relate to our problem statement:

What is the picture of digital connectivity and engagement in 2020?

What is the effect of the COVID-19 pandemic on online and distance learning, and how might this also evolve in the future?

How does student engagement with different types of education technology change over the course of the pandemic?

How does student engagement with online learning platforms relate to different geography? Demographic context (e.g., race/ethnicity, ESL, learning disability)? Learning context? Socioeconomic status?

Do certain state interventions, practices or policies (e.g., stimulus, reopening, eviction moratorium) correlate with the increase or decrease online engagement?

Import

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Read

In [ ]:
product = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
product

In [ ]:
district = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")
district

Check for null values

In [ ]:
product.isnull().sum()

In [ ]:
district.isnull().sum()

Impute null values

In [ ]:
for column in product.columns:
    product[column].fillna(product[column].mode()[0], inplace=True)
product

In [ ]:
for column in district.columns:
    district[column].fillna(district[column].mode()[0], inplace=True)
district

Analysis of product

Function to split multiple values to different rows for Sector(s)

In [ ]:
df = product
target_column = product['Sector(s)']

In [ ]:
def splitDataFrameList(df,target_column,separator):
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s.strip()
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

product = splitDataFrameList(product, 'Sector(s)', ';')
product

In [ ]:
product.groupby('Provider/Company Name')['Provider/Company Name'].count().sort_values(ascending=False)

Google and Microsoft are the two companies that provide the most educational serves to the states covered in the data given.

Sector(s)

In [ ]:
product.groupby('Sector(s)')['Sector(s)'].count().sort_values(ascending=False)

The sectors covered were primarily PreK to 12, followed by Higher Ed, and lastly thr Corporate sector.

In [ ]:
sectors_data = product.groupby('Sector(s)').size().reset_index(name='count')
sectors = sectors_data['Sector(s)'].values
plt.pie(sectors_data['count'], labels=sectors, autopct='%1.1f%%', startangle=15, explode=None)
plt.title('% of Digital Learning Products per Education Sectors')
plt.axis('equal')
plt.show()


List unique values in Primary Essential Function

In [ ]:
list(product['Primary Essential Function'].unique())

Split PEF into PSF Main Category

In [ ]:
product['PSF Main Category'] = product['Primary Essential Function'].apply(lambda x: x.split()[0]) 

Graph Primary Essential Function

In [ ]:
product.groupby('PSF Main Category')['PSF Main Category'].count().sort_values(ascending=False)

The PSF Main Category is initially LC, followed by CM, LCO, and finally LC/CM/SDO.

In [ ]:
pef_data = product.groupby('PSF Main Category').size().reset_index(name='count')
category  = pef_data['PSF Main Category'].values
plt.pie(pef_data['count'], labels=category, autopct='%1.1f%%', startangle=15, explode=None)
plt.title('% of Digital Learning Products per Primary Essential Function')
plt.axis('equal')
plt.show()


Plot PSF

In [ ]:
sns.set(rc={'figure.figsize':(15,15)})
sns.catplot(x="PSF Main Category", kind="count", hue="Sector(s)", data=product, height=8, aspect=11/8)
plt.show()

Extract subcategories of PEF

In [ ]:
import re

product['PSF Sub Categories'] = product['Primary Essential Function'].apply(lambda x: re.split("^(LC - |CM - |SDO - |LC\/CM\/SDO - )", x)[-1])


Group PSF

In [ ]:
psf_groups = product.groupby(['PSF Main Category', 'PSF Sub Categories', ]).size().reset_index(name='count')
psf_groups


Analysis of district

Plot distribution of locale

In [ ]:
sns.catplot(x='locale', kind='count', data=district, height=5, aspect=5/5)
plt.title("Distribution of locale")
plt.show()

Count localities

In [ ]:
district.groupby('locale').locale.count().sort_values(ascending=False)

The locale is initially Suburb, followed by Rural, City, and then Town.

Count state

In [ ]:
district.groupby('state').state.count().sort_values(ascending=False)

There are six states that appear to get most of the educational support, being Conneticut,Utah, Massechusettes, Illinois, California and Ohio. 

In [ ]:
sns.catplot(x='state', kind='count', data=district, height=5, aspect=5/5)
plt.title("State")
plt.show()

In [ ]:
pef_data = district.groupby('state').size().reset_index(name='count')
category  = pef_data['state'].values
plt.pie(pef_data['count'], labels=category, autopct='%1.1f%%', startangle=15, explode=None)
plt.title('States represented')
plt.axis('equal')
plt.show()


Internet connection

In [ ]:
district.groupby('county_connections_ratio')['county_connections_ratio'].count().sort_values(ascending=False)

In [ ]:
sns.catplot(x='county_connections_ratio', kind='count', data=district, height=5, aspect=5/5)
plt.title("Internet connection ratio")
plt.show()

Analyse percent black and hispanic

In [ ]:
district.groupby('pct_black/hispanic')['pct_black/hispanic'].count().sort_values(ascending=False)

In [ ]:
sns.catplot(x='pct_black/hispanic', kind='count', data=district, height=5, aspect=5/5)
plt.title("Percentage black and hispanic")
plt.show()

Percentage of free or reduced meals

In [ ]:
district.groupby('pct_free/reduced')['pct_free/reduced'].count().sort_values(ascending=False)

In [ ]:
sns.catplot(x='pct_free/reduced', kind='count', data=district, height=5, aspect=5/5)
plt.title("Percentage of free or reduced meals")
plt.show()

Expendature per pupil

In [ ]:
district.groupby('pp_total_raw')['pp_total_raw'].count().sort_values(ascending=False)

In [ ]:
sns.catplot(x='pp_total_raw', kind='count', data=district, height=5, aspect=5/5)
plt.title("Expense per pupil")
plt.show()

Analysis of data

OECD's publication
see:- https://www.oecd.org/education/the-impact-of-covid-19-on-education-insights-education-at-a-glance-2020.pdf

The Organisation for Economic Cooperation and Development has published a paper, entitled "The Impact of COV!( on Education Insights: Education at a Glance 2020", that is well worth reading to gain insight on how the process of education has been affected since the COV19 outbreak. 

The highlights of this paper illustrate what efforts countries have made to continue to educate their population despite the severe restrictions imposed by lockdowns:-

Countries used remote learning, such as radio and television education, instructional packages, and online instructional resources.

Online education consisted of:- educational content, virtual lessons, online support for parents and pupils, and self-paced formalised lessons.

Teachers have played a critical role in the education process:-

- 60% of teachers received ICT training.

- 18% of teachers reported a higher need for ICT training.

- 36% of teachers reported participating in online courses or siminars.

- Teachers are not relying heavily on distance learning for their development.

What is the picture of digital connectivity and engagement in 2020?

see:- https://www.theguardian.com/world/2020/apr/13/coronavirus-covid-19-exposes-cracks-us-digital-divide

The Guardian wrote a very insightful article, "US's digital divide 'is going to kill eople' as COV19 exposes inequalities". Exclusive research shows drop in connectivity is impacting rural and urban areas with populations already underserved by the medical system or racked with poverty.

With most of the country on lockdown and millions relying on the internet for work, healthcare, education and shopping, research by M-Lab, an open source project which monitors global internet performance, showed that internet service slowed across the country after the lockdowns

In March 2020, most people in 62% of counties across the US did not have the government’s minimum download speed for broadband internet, according to M-Lab. Those communities that are underserved are not going to be able to transition to an online workplace or school environment.

The Federal Communications Commission (FCC) says 21 million Americans lack high-speed internet access but other studies have estimated the number at close to 42 million. To address that gap in connectivity, students without access to internet have been forced to roam their home towns looking for open wifi networks. Facing a remote learning crisis, some school districts are paying for their students’ internet to make remote learning possible.

The statistics presented in this competition only covered 23 states, which is less than half of the 50 US states. Conneticut, Utah, Massachusettes, Illinois, California, and Ohio have received the greatest representation in this competition.



What is the effect of the COVID-19 pandemic on online and distance learning, and how might this also evolve in the future?

see:- https://www.oecd.org/coronavirus/policy-responses/strengthening-online-learning-when-schools-are-closed-the-role-of-families-and-teachers-in-supporting-students-during-the-covid-19-crisis-c4ecba6c/

The Organisation for Economic Cooperation and Development has written a policy brief on their website, which is worth looking at:-

The COVID-19 crisis has forced education systems worldwide to find alternatives to face-to-face instruction. As a result, online teaching and learning have been used by teachers and students on an unprecedented scale.

The findings of the OECD include the following statements:-

The COVID-19 crisis has obliged most education systems to adopt alternatives to face-to-face teaching and learning. Many education systems moved activities online, to allow instruction to continue despite school closures. 

There are concerns that online learning may have been a sub-optimal substitute for face-to-face instruction, especially so in the absence of universal access to infrastructure (hardware and software) and lack of adequate preparation among teachers and students for the unique demands that online teaching learning pose.

Students’ attitudes and dispositions are influenced to a great degree by the support they receive from families and teachers and by the role models they are exposed to. Different forms of support from families and teachers, including parental emotional support and teacher enthusiasm, are found to be important for the development of positive attitudes towards learning and can ensure that students acquire the attitudes and dispositions that can maximise their ability to make the most of online learning opportunities.

The recommendations of the OECD include:-

Education systems should aim to strengthen engagement between schools and parents in order to improve information and guidance to parents on effective practices for supporting their children’s learning. At the same time, teachers need support to incorporate technology effectively into their teaching practices and methods and help students overcome some of the difficulties that are associated with this form of learning environment.



How does student engagement with different types of education technology change over the course of the pandemic?

see:- https://www.frontiersin.org/articles/10.3389/fcomm.2021.641865/full

Frontiers in Communication has written an opinion piece that discussed student engagement in a post covid world:-

When the World Health Organisation (WHO) announced the world was in a pandemic in March 2020, institutions of learning were force to change the way they educated their students. This change in learning methods was primarily focused on converting traditional face-to-face in person lectures to digital online training on most, if not all, courses.

There are three types of student engagement, being learner to learner, learner to instructor, and learner to content.

Learner to learner engagements involves access to other students through means such asonline discussion boards. When students are allowed to communicate with each other it gives them a feeling of inclusion so they have the opportunity to learn experiementally.

Learner to instructor engagement involves the communication the student has with the course instructor and is the key point as to whether the student gains a sense of belonging and will potentially do well in the course.

Learning to course content engagement involves the time the student spends looking at the course material, such as textbooks, video, audio and interactive games. It goes without saying that the more time a student spends studying the course material and working on lesson assignments, the greater his propensity to succeed in the learning environment.

Findings suggest that students and faculty both agreed on the three types of engagement. In order to build the best couse in a post_COV19 environment, all three methods of engagement need to be incorporated into the lesson plan.



How does student engagement with online learning platforms relate to different geography? Demographic context (e.g., race/ethnicity, ESL, learning disability)? Learning context? Socioeconomic status?

see:- https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7308790/

The NCBI website published an article entitled, "Online Learning: A Panacea in the Time of COVID-19 Crisis". This article focused on the fact that the world was forced to change the way it educates students in the wake of the COV19 pandemic. Out of necessity, most learning establishments have moved to online learning because face-to-face learning is not feasible when students and teachers alike are required to comply with social distancing measures.

Accessibility, affordability, flexibility, learning pedagogy, life long learning, and policy are some of the arguments related to online learning. It is said that online learning is easily accessible and can even reach rural and remote areas. It is considered to be a relatively cheaper mode of education in terms of the lower cost of transportation, accommodation, and the overall cost of institution-based learning. Flexibility is another interesting aspect of online learning, as a learner can plan their time for completion of courses available online. 

Combining face-to-face lectures with technology gives rise to type of hybrid learning. This type of learning environment can increase the learning potential of the students because they can learn anytime and anywhere, thereby developing new skills in the process leading to life-long learning. 

Do certain state interventions, practices or policies (e.g., stimulus, reopening, eviction moratorium) correlate with the increase or decrease online engagement?

see:- https://www.chalkbeat.org/2021/3/12/22328181/schools-stimulus-money-questions

There is a website called chalkbeat, which reports on educational matters acriss the US. This website has attempted to answer questions concerning the stimulus package and how it will affect the school system:-

President Biden signed the USD 1.9 trillion American Rescue Plan into law. The package includes close to USD 130 billion for K-12 education, as well as a number of other provisions that will affect schools.

The stimulus money can be used for:-

Cleaning supplies

Education technology
Mental health services

Summer or after-school programs

School building improvements that “reduce risk of virus transmission and exposure to environmental health hazards”
Implementation of public health guidance

“Other activities that are necessary to maintain the operation of and continuity of services ... and continuing to employ existing staff.”

“Activities to address the unique needs of low-income children or students, children with disabilities, English learners, racial and ethnic minorities, students experiencing homelessness, and foster care youth.”

School districts must use at least 20% of the money to address learning loss. 

In addition to the direct money given to schools, the bill includes USD 7 billion to create an Emergency Connectivity Fund through a program known as E-Rate, which subsidizes internet access for schools and libraries. Schools that participate in the program can use the new money to help students and teachers access the internet at home.

The bill temporarily expands direct payments to families by boosting the amount from USD 2,000 per child to USD 3,000 (or USD 3,600 for children under the age of 6). It also makes very low-income families eligible for this program for the first time. The bill also provides direct payments of USD 1,400 to most individuals, and extends increases in unemployment benefits. Combined, these measures are projected to reduce the share of children who fall below the federal poverty line (which is set at USD 26,500 for a family of four) — from 13.7% to 6.5%. Black and Hispanic children are likely to see the biggest declines in poverty.

Solution

I think much of the solution to this competition question has been resolved because of the American Rescue Plan is intended to give much needed funding to schools and parents can receive some of the funding directly.

I believe that we are swiftly moving into a digital era and children who do not have access to a tablet and/or a laptop will be left behind. It is for that reason that schools should use the funding they receive to issie laptop computers to those children whose parents cannot afford to buy one, and this should be a means tested benefit.

I believe that the best way to move forward with regard to education is to have a hybrid arrangement. Some of the courses will be taught on line while a few others will be given face to face. 

With so many courses being taught online, there needs to be a mechanism for students to mix with other students, and these events could very well be aligned with the face to face lessons.